In [ ]:
!pip install transformers pandas pandasql -q


  Preparing metadata (setup.py) ... done


In [ ]:
import json
import pandas as pd

with open('/content/company_data_dirty.json') as f:
    data = json.load(f)

df = pd.DataFrame(data['Emp'])
df = df.replace('N/A', pd.NA).dropna()
df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
df['doj'] = pd.to_datetime(df['doj'], errors='coerce')
df = df.dropna().reset_index(drop=True)


In [ ]:
import random

genders = ['Male', 'Female']
countries = ['India', 'Peru', 'Brazil', 'USA', 'Germany', 'France', 'Canada', 'Japan', 'Mexico', 'Nigeria', 'Australia', 'Spain', 'Argentina']
names = ['John', 'Maria', 'Carlos', 'Ali', 'Fatima', 'Liam', 'Emma']

questions = []
sqls = []

for _ in range(1000):
    q_type = random.choice(['gender_country', 'dob_before', 'doj_after', 'count_gender', 'name_match'])
    if q_type == 'gender_country':
        gender = random.choice(genders)
        country = random.choice(countries)
        questions.append(f"Show all {gender.lower()} employees from {country}")
        sqls.append(f"SELECT * FROM employees WHERE gender = '{gender}' AND country = '{country}'")
    elif q_type == 'dob_before':
        year = random.randint(1960, 2000)
        questions.append(f"List employees born before {year}")
        sqls.append(f"SELECT * FROM employees WHERE dob < '{year}-01-01'")
    elif q_type == 'doj_after':
        year = random.randint(2010, 2023)
        questions.append(f"Show employees who joined after {year}")
        sqls.append(f"SELECT * FROM employees WHERE doj > '{year}-01-01'")
    elif q_type == 'count_gender':
        gender = random.choice(genders)
        questions.append(f"How many {gender.lower()} employees are there?")
        sqls.append(f"SELECT COUNT(*) FROM employees WHERE gender = '{gender}'")
    elif q_type == 'name_match':
        name = random.choice(names)
        questions.append(f"Find employees named {name}")
        sqls.append(f"SELECT * FROM employees WHERE name = '{name}'")

train_df = pd.DataFrame({'question': questions, 'sql': sqls})


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

class SQLDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.data = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = "translate English to SQL: " + self.data.iloc[idx]["question"]
        target_text = self.data.iloc[idx]["sql"]
        input_enc = tokenizer(input_text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
        target_enc = tokenizer(target_text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
        return {
            "input_ids": input_enc["input_ids"].squeeze(),
            "attention_mask": input_enc["attention_mask"].squeeze(),
            "labels": target_enc["input_ids"].squeeze()
        }

train_dataset = SQLDataset(train_df, tokenizer)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from transformers import Trainer, TrainingArguments
import os

os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir="./t5_sql_model",
    per_device_train_batch_size=4,
    num_train_epochs=10,
    save_strategy="no",
    logging_steps=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

trainer.train()



Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-11-1379537606.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
2,13.521300
4,11.436600
6,9.828400
8,8.928200
10,10.373800
12,8.954400
14,7.933100
16,6.724200
18,5.846600
20,5.127600


KeyboardInterrupt: 

In [ ]:
def generate_sql(q):
    input_text = "translate English to SQL: " + q
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output = model.generate(input_ids, max_new_tokens=128)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
generate_sql("How many female employees are there?")


"SELECT COUNT(*) FROM employees WHERE gender = 'Female'"